In [1]:
import re
import pandas as pd
import numpy as np

<head><h4>Functions:</h4></head><BR>
<div>*fix_genre - receives the dataframe with the unorganized genres column, creates a column for each genre from a list, runs a regex on the existing rows to seperate the genres that start with a capital letter, after that I mark the column of each genre with 1's</div><BR>
    <div>*numerize_col - receives the dataframe and a column name, turns said column from categorical to numerical value</div>

In [2]:
def fix_genre(df):
    pattern = '[A-Z][^A-Z]*'
    genre_list = ['Action','Adventure','Comedy','Drama','Ecchi','Fantasy', 'Horror', 'Mahou Shoujo', 'Mecha', 'Music','Mystery'
                 ,'Psychological', 'Romance','Sci-Fi', 'Slice of Life', 'Sports','Supernatural','Thriller']
    for genre in genre_list:
        df[genre] = 0
    row_index = 0
    for genre in df['Genres']:
        check_list = re.findall(pattern, genre)
        for check in check_list:
            if(check == None or check == 'Shoujo' or check =='Fi' or check=='Life'):
                continue
            if(check == 'Mahou '):
                check += 'Shoujo'
            if(check == 'Sci-'):
                check += 'Fi'
            if(check == 'Slice of '):
                check += 'Life'
            if(check in df.columns):
                df.at[row_index, check] = 1
        row_index +=1
    return df

In [3]:
def numerize_col(df, col_name):
    value_list, unique_list = df[col_name].factorize()
    df[col_name] = value_list
    return df

<h4>Main Body:</h4>
    <p>The main purpose of the body of the function is the cleaning of the dataframe and converting none numerical values, I start by getting the dataframes from two csvs and combining them into one dataframe, after that, I start by dropping unnecessary columns or those with too many missing values. in the next stage after checking which rows has too many missing values I get rid of them aswell. I fill the missing 'Source' rows with 'Other'.
    
after cleaning the dataframe of missing values, I reset the indexes, and send the dataframe to fix_genre function to get a new dataframe with organized genre columns, after that I drop the old 'Genres' column.

afterwards I clean 'Mean Score' and 'Format' of unnecessary strings and convert them to numerical values using numerize_col function.

Lastly I convert all columns from whatever dtypes they were to int64 and save the clean dataframe in a new .csv file.</p>

In [4]:
df_1 = pd.read_csv('./Anime_Data_Frame_1')
df_2 = pd.read_csv('./Anime_Data_Frame_2')

In [24]:
complete_df = df_1.append(df_2)

<div>Dropping unnecessary columns</div>

In [6]:
drop_list = ['Unnamed: 0','Average Score','Duration', 'English','EpisodeDuration','Episodes','Studios','Hashtag','Synonyms', 'Producers','Status','Romaji','Native']
col_dropped_df = complete_df.drop(drop_list, axis = 1)

In [7]:
col_dropped_df

,Format,Mean Score,Popularity,Favorites,Source,Genres
0,NaN,NaN,NaN,NaN,NaN,NaN
1,TV,66%,6050.0,69.0,Manga,ComedyDramaFantasyMahou ShoujoRomance
2,TV,72%,6307.0,98.0,Manga,ActionAdventureDramaSci-Fi
3,Special,59%,103.0,NaN,NaN,ComedySupernatural
4,TV Short,60%,6176.0,48.0,Manga,ComedyMechaSci-FiSlice of Life
...,...,...,...,...,...,...
5681,TV,NaN,117.0,1.0,Video Game,Mecha
5682,Music,74%,67.0,NaN,Original,Supernatural
5683,Music,76%,194.0,3.0,Original,NaN
5684,Music,76%,225.0,4.0,Original,NaN


<div>Checking null values, and dropping rows accordingly.

I prefer to fill Source rather than dropping those extra rows because all unknown values can be filled with 'Other'</div>

In [8]:
col_dropped_df.isnull().sum()

Format         415
Mean Score    1023
Popularity     361
Favorites     5011
Source        2645
Genres        2315
dtype: int64

In [9]:
row_drop_list = ['Mean Score','Popularity','Favorites','Genres']
preprocessed_df = col_dropped_df.dropna(axis = 0, subset = row_drop_list)


In [10]:
preprocessed_df.isnull().sum()

Format           0
Mean Score       0
Popularity       0
Favorites        0
Source        1011
Genres           0
dtype: int64

In [11]:
preprocessed_df['Source'].fillna('Other', inplace = True)
preprocessed_df.isnull().sum()

C:\Users\ItanMallul\anaconda3\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Format        0
Mean Score    0
Popularity    0
Favorites     0
Source        0
Genres        0
dtype: int64

In [12]:
preprocessed_df['Source'].unique()

array(['Manga', 'Light Novel', 'Other', 'Original', 'Novel', 'Comic',
       'Video Game', 'Web Novel', 'Visual Novel', 'Picture Book',
       'Multimedia Project', 'Doujinshi', 'Anime', 'Game', 'Live Action'],
      dtype=object)

In [13]:
preprocessed_df.isnull().sum()

Format        0
Mean Score    0
Popularity    0
Favorites     0
Source        0
Genres        0
dtype: int64

In [25]:
preprocessed_df.reset_index(drop=True, inplace=True)
preprocessed_df['Format'].unique()

array(['TV', 'TV Short', 'OVA', 'Movie', 'Special', 'ONA',
       'ONA\r\n(Chinese)', 'Music', 'TV\r\n(Chinese)',
       'Movie\r\n(South Korean)', 'Movie\r\n(Chinese)',
       'TV\r\n(South Korean)', 'ONA\r\n(South Korean)',
       'OVA\r\n(South Korean)', 'TV Short\r\n(South Korean)',
       'ONA\r\n(Taiwanese)', 'Movie\r\n\r\n(South Korean)',
       'TV Short\r\n(Chinese)', 'Movie\n(Chinese)', 'ONA\n(Chinese)',
       'TV Short\n(South Korean)', 'Movie\n(South Korean)',
       'TV\n(Chinese)', 'Movie\n(Taiwanese)', 'TV Short\n(Chinese)',
       'TV\n(South Korean)', 'ONA\n\n(Chinese)', 'ONA\n(South Korean)',
       'ONA\n(Doujin)', 'TV\n(Taiwanese)', 'OVA\n(Chinese)',
       'ONA\n(Taiwanese)', 'ONA\n (South Korean)', 'ONA\n (Chinese)',
       'OVA\n(Doujin)', 'Movie\n(Doujin)'], dtype=object)

In [28]:
format_dict = {k: v for v, k in enumerate(preprocessed_df['Format'].unique())}

In [29]:
format_dict ##to remember each formats number

{'TV': 0,
 'TV Short': 1,
 'OVA': 2,
 'Movie': 3,
 'Special': 4,
 'ONA': 5,
 'ONA\r\n(Chinese)': 6,
 'Music': 7,
 'TV\r\n(Chinese)': 8,
 'Movie\r\n(South Korean)': 9,
 'Movie\r\n(Chinese)': 10,
 'TV\r\n(South Korean)': 11,
 'ONA\r\n(South Korean)': 12,
 'OVA\r\n(South Korean)': 13,
 'TV Short\r\n(South Korean)': 14,
 'ONA\r\n(Taiwanese)': 15,
 'Movie\r\n\r\n(South Korean)': 16,
 'TV Short\r\n(Chinese)': 17,
 'Movie\n(Chinese)': 18,
 'ONA\n(Chinese)': 19,
 'TV Short\n(South Korean)': 20,
 'Movie\n(South Korean)': 21,
 'TV\n(Chinese)': 22,
 'Movie\n(Taiwanese)': 23,
 'TV Short\n(Chinese)': 24,
 'TV\n(South Korean)': 25,
 'ONA\n\n(Chinese)': 26,
 'ONA\n(South Korean)': 27,
 'ONA\n(Doujin)': 28,
 'TV\n(Taiwanese)': 29,
 'OVA\n(Chinese)': 30,
 'ONA\n(Taiwanese)': 31,
 'ONA\n (South Korean)': 32,
 'ONA\n (Chinese)': 33,
 'OVA\n(Doujin)': 34,
 'Movie\n(Doujin)': 35}

In [15]:
genred_df = fix_genre(preprocessed_df)
genred_df = genred_df.drop('Genres', axis = 1)

C:\Users\ITANMA~1\AppData\Local\Temp/ipykernel_1572/1151312368.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[genre] = 0


<p>'Mean Score' and 'Format' have unnecessary characters that are in the way of numerizing the values, so we will clean them up before turning them into numbers.</p>

In [16]:
genred_df['Mean Score'] = genred_df['Mean Score'].str.replace("%","")
genred_df['Format'] = genred_df['Format'].str.replace("/n","").replace("\n","")

In [22]:
preprocessed_df['Format'].unique()

36

In [18]:
numerized_df = numerize_col(genred_df, 'Source')
numerized_df = numerize_col(numerized_df, 'Format')
numerized_df

,Format,Mean Score,Popularity,Favorites,Source,Action,Adventure,Comedy,Drama,Ecchi,...,Mecha,Music,Mystery,Psychological,Romance,Sci-Fi,Slice of Life,Sports,Supernatural,Thriller
0,0,66,6050.0,69.0,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
1,0,72,6307.0,98.0,0,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,1,60,6176.0,48.0,0,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,0
3,0,61,43462.0,656.0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
4,2,76,58959.0,797.0,1,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9522,2,61,22.0,1.0,3,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9523,5,62,73.0,2.0,3,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9524,4,61,71.0,2.0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9525,5,66,159.0,2.0,3,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [19]:
numerized_df = numerized_df.astype({'Favorites': 'int64', 'Popularity': 'int64', 'Mean Score': 'int64'}, errors='raise')
numerized_df.dtypes

Format           int64
Mean Score       int64
Popularity       int64
Favorites        int64
Source           int64
Action           int64
Adventure        int64
Comedy           int64
Drama            int64
Ecchi            int64
Fantasy          int64
Horror           int64
Mahou Shoujo     int64
Mecha            int64
Music            int64
Mystery          int64
Psychological    int64
Romance          int64
Sci-Fi           int64
Slice of Life    int64
Sports           int64
Supernatural     int64
Thriller         int64
dtype: object

In [20]:
numerized_df

,Format,Mean Score,Popularity,Favorites,Source,Action,Adventure,Comedy,Drama,Ecchi,...,Mecha,Music,Mystery,Psychological,Romance,Sci-Fi,Slice of Life,Sports,Supernatural,Thriller
0,0,66,6050,69,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
1,0,72,6307,98,0,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,1,60,6176,48,0,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,0
3,0,61,43462,656,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
4,2,76,58959,797,1,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9522,2,61,22,1,3,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9523,5,62,73,2,3,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9524,4,61,71,2,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9525,5,66,159,2,3,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [21]:
numerized_df.to_csv('clean_anime_dataframe', index_label = False)